# Data Acquisition from Web available Data

Sites that don't offer download options of the whole database were scraped. The code is available in scraped_data.ipynb

In [84]:
import pandas as pd

In [35]:
class converters:

  def __init__(self):
    self.name = 'hi'

  def conv_APD_sequence_release_09142020(self, l, infile):
    names, rest = l.split(' ', 1)# ( at times ( ( at ttimes [])))
    id, name = names.strip('>').split('|')
    seq = {'id':id, 'name':name, 'AB':1, 'description':rest, 'OX':'alien', 'dataset':infile, 'seq':''}
    return name, seq
  
  def conv_dbaasp_nonribosomal(self, l, infile):
    l = l.split('> ')[1]
    id, name = l.strip('\n').split(' ', 1)
    seq = {'id':id, 'name':name, 'AB':1, 'description':'None', 'OX':'alien', 'dataset':infile, 'seq':''}
    return name, seq
  
  def conv_dbaasp_ribosomal(self, l, infile):
    l = l.split('> ')[1]
    id, name = l.strip('\n').split(' ', 1)
    seq = {'id':id, 'name':name, 'AB':1, 'description':'None', 'OX':'alien', 'dataset':infile, 'seq':''}
    return name, seq
  

  def conv_BAAMPs_data(self, infile):
    df = pd.read_csv(infile)

    dfnew = df[['PeptideID', 'PeptideName', 'Microorganism group', 'Microorganism', 'PeptideSequence']]
    dfnew.insert(2, 'AB', [1 for i in range(dfnew.shape[0]) ] )
    dfnew.insert(5, 'dataset', [ 'BAAMPs_data.csv' for i in range(dfnew.shape[0]) ] )
    colnames = ['id', 'name', 'AB', 'description', 'OX', 'dataset', 'seq']
    dfnew.columns = colnames
    dfnew.index = dfnew.name

    return dfnew

  def conv_dramp_antimicrobial(self, l, infile):
    id = l.strip('> ').strip('\n')
    name = f'dramp_{id}'# cant set to None as messes with indexation of pd frame
    seq = {'id':id, 'name':name, 'AB':1, 'description':'None', 'OX':'alien', 'dataset':infile, 'seq':''}
    return name, seq

  def conv_LAMP2(self, l, infile):
      l = l.split('>')[1]
      id, rest = l.strip('\n').split('|', 1)
      name = f'LAMP2_{id}'
      seq = {'id':id, 'name':name, 'AB':1, 'description':rest, 'OX':'alien', 'dataset':infile, 'seq':''}
      return name, seq


  def conv_InverPep(self, l, infile):
      l = l.strip('>').strip('\n')
      ls = l.split('|', 1)
      if len(ls)==1: id, name = [ls[0], f'InverPep_{ls[0]}' ]
      if len(ls)==2: id, name = ls#[0], name = ls[1]
      if name =='-': name += id # else get same index, overwrite datarows
      seq = {'id':id, 'name':name, 'AB':1, 'description':'None', 'OX':'alien', 'dataset':infile, 'seq':''}
      return name, seq
    

  def conv_enzybase(self, l, infile):
        l = l.strip('>')
        name, some, id, rest = l.strip('\n').split('|')
        seq = {'id':id, 'name':name, 'AB':1, 'description':f'type_{some}_'+rest, 'OX':'alien', 'dataset':infile, 'seq':''}
        return name, seq

In [24]:
def tocsv(infile, save, ofile=''):
    
  print('input file', infile)

  filetype = infile.split('.fasta')[0].split('.txt')[0].split('.csv')[0]
  converter = converters()
  convfct = getattr(converter, 'conv_'+str(filetype))

  if infile.split('.')[1] == 'csv':
    df = convfct(infile)
  
  else:

    with open(infile, 'r') as inf:
      
      print('converting with fct: ', convfct)
      
      seqs = {}
      lastname = 'None'

      flines = inf.readlines()

      print(flines[:4])

      for l in flines:
        if l[0]=='>':# sequence identifier
          name, seq = convfct(l, infile)
          seqs[name] = seq
          lastname = name

        else:
          seq = seqs[lastname]
          seq['seq'] += l.strip('>').strip('\n').strip(' ')

    inf.close()

    df = pd.DataFrame.from_dict(seqs).transpose()
    lens = df.seq.map(len)
    print('min, max seq: ', min(lens), max(lens))

  if save:
      destin = ofile.split('.csv')[0]+'_out.csv'
      df.to_csv(destin, index = False )
      print('saved df as csv to:', destin)
      
  return df

# Databases from Table 2

seq, T/F, database, (additional info, species, assay method)

In [7]:
# http://aps.unmc.edu/AP/main.php but now cant access
df = tocsv('APD_sequence_release_09142020.fasta', True, 'APD_sequence_release_09142020')
df.head(2)

input file APD_sequence_release_09142020.fasta
converting with fct:  <bound method converters.conv_APD_sequence_release_09142020 of <__main__.converters object at 0x7fd306c11c60>>
min, max seq:  4 183
saved df as csv to: APD_sequence_release_09142020_out.csv


,id,name,AB,description,OX,dataset,seq
Dermaseptin-B2,00001,Dermaseptin-B2,1,"(XXA, DRS-B2, Dermaseptin B2, DRS B2, DS bII,...",alien,APD_sequence_release_09142020.fasta,GLWSKIKEVGKEAAKAAAKAAGKAALGAVSEAV
Abaecin,00002,Abaecin,1,"(Pro-rich; insects, arthropods, invertebrates,...",alien,APD_sequence_release_09142020.fasta,YVPLPNVPQPGRRPFPTFPGQGPFNPKIKWPQGY


In [ ]:
#TODO http://www.camp3.bicnirrh.res.in/ length: -200 validation: experiemntal
# not loading and if, cant download list, newer version http://camp.bicnirrh.res.in/ not accessible

In [32]:
# https://dbaasp.org
# downloaded seq lengths from -200, nonribosomal

df = tocsv('dbaasp_nonribosomal.txt', True, 'dbaasp_nonribosomal')
df.head(2)

input file dbaasp_nonribosomal.txt
converting with fct:  <bound method converters.conv_dbaasp_nonribosomal of <__main__.converters object at 0x7fd30bea52d0>>
['> DBAASPN_1688 Gramicidin S, GS\n', 'VXLfPVXLfP\n', '> DBAASPN_3314 Gratisin, GR\n', 'VXLfPyVXLfPy\n']
min, max seq:  1 22
saved df as csv to: dbaasp_nonribosomal_out.csv


,id,name,AB,description,OX,dataset,seq
"Gramicidin S, GS",DBAASPN_1688,"Gramicidin S, GS",1,None,alien,dbaasp_nonribosomal.txt,VXLfPVXLfP
"Gratisin, GR",DBAASPN_3314,"Gratisin, GR",1,None,alien,dbaasp_nonribosomal.txt,VXLfPyVXLfPy


In [100]:
# https://dbaasp.org
# downloaded seq lengths from -200, ribosomal

df = tocsv('dbaasp_ribosomal.txt', True, 'dbaasp_ribosomal')
df.head(2)

input file dbaasp_ribosomal.txt
converting with fct:  <bound method converters.conv_dbaasp_ribosomal of <__main__.converters object at 0x7fd2ed798850>>
['> DBAASPR_11 Cathelicidin-1, CATH-1, Fowlicidin-1\n', 'RVKRVWPLVIRTVIAGYNLYRAIKKK\n', '> DBAASPR_14 Brevinin-2 related peptide B2RP   \n', 'GIWDTIKSMGKVFAGKILQNL\n']
min, max seq:  4 190
saved df as csv to: dbaasp_ribosomal_out.csv


,id,name,AB,description,OX,dataset,seq
"Cathelicidin-1, CATH-1, Fowlicidin-1",DBAASPR_9526,"Cathelicidin-1, CATH-1, Fowlicidin-1",1,None,alien,dbaasp_ribosomal.txt,RVKRVWPLVIRTVIAGYNLYRAIKKK
Brevinin-2 related peptide B2RP,DBAASPR_14,Brevinin-2 related peptide B2RP,1,None,alien,dbaasp_ribosomal.txt,GIWDTIKSMGKVFAGKILQNL


In [ ]:
#TODO http://defensins.bii.a-star.edu.sg/
# website currently under maintenance

In [36]:
#http://www.baamps.it/
# download csv in browse /Export, all filter later
# TODO! duplicates in names ?and sequence but diff origins

df = tocsv('BAAMPs_data.csv', True, 'BAAMPs_data')
df.head(3)

input file BAAMPs_data.csv
saved df as csv to: BAAMPs_data_out.csv


,id,name,AB,description,OX,dataset,seq
name,,,,,,,
Pleurocidin,1,Pleurocidin,1,gram neg,Escherichia coli ATCC 25922,BAAMPs_data.csv,GWGSFFKKAAHVGKHVGKAALTHYL
Pleurocidin,1,Pleurocidin,1,gram neg,Pseudomonas aeruginosa ATCC 27853,BAAMPs_data.csv,GWGSFFKKAAHVGKHVGKAALTHYL
Pleurocidin,1,Pleurocidin,1,gram pos,Staphylococcus aureus ATCC 25923,BAAMPs_data.csv,GWGSFFKKAAHVGKHVGKAALTHYL


In [ ]:
#TODO http://bactibase.hammamilab.org/about.php
# nothing? showing

In [ ]:
# http://split4.pmfst.hr/dadp/
# scrape 

In [46]:
# http://dramp.cpu-bioinfor.org selected antimicorbial dataset

df = tocsv('dramp_antimicrobial.fasta', True, 'dramp_antimicrobial')
df.head(2)

input file dramp_antimicrobial.fasta
converting with fct:  <bound method converters.conv_dramp_antimicrobial of <__main__.converters object at 0x7fd30cc79690>>
min, max seq:  1 101
saved df as csv to: dramp_antimicrobial_out.csv


,id,name,AB,description,OX,dataset,seq
dramp_DRAMP00005,DRAMP00005,dramp_DRAMP00005,1,None,alien,dramp_antimicrobial.fasta,SLGPAIKATRQVCPKATRFVTVSCKKSDCQ
dramp_DRAMP00017,DRAMP00017,dramp_DRAMP00017,1,None,alien,dramp_antimicrobial.fasta,VTSWSLCTPGCTSPGGGSNCSFCC


In [ ]:
# https://peptaibol.cryst.bbk.ac.uk/introduction.htm
# https://peptaibol.cryst.bbk.ac.uk/peptaibol_database_1lettercodes.htm will have to scrape

In [110]:
# http://biotechlab.fudan.edu.cn/database/lamp/index.php LAMP2
# XML cant read, some more info (crosslinking to other db), later dropping patent = synthetic

""" # options didnt work
import codecs
with codecs.open('LAMP2.XML', 'r', encoding='iso-8859-1') as f:
    content = f.read()
with codecs.open('LAMP2.xml', 'w', encoding='utf-8') as f:
    f.write(content)

import xml.etree.ElementTree as ET
root = ET.parse('LAMP2.XML').getroot()
"""

df = tocsv('LAMP2.fasta', True, 'LAMP2')
df.head(2)

input file LAMP2.fasta
converting with fct:  <bound method converters.conv_LAMP2 of <__main__.converters object at 0x7fd2d0f80070>>
['>L01A000872|Sequence 82 from patent US 7166769\n', 'AAEDSQVGEGVVKIDCGGRCKGRCSKSSRPNLCLRACNSCCYRCNCVPPGTAGNHHLCPCYASITTRGGRLKCP\n', '>L02A000612|Chrombacin \n', 'AAEFPDFYDSEEQMGPHQEAEDEKDRADQRVLTEEEKKELENLAAMDLELQKIAEKFSQR\n']
min, max seq:  2 99
saved df as csv to: LAMP2_out.csv


,id,name,AB,description,OX,dataset,seq
LAMP2_L01A000872,L01A000872,LAMP2_L01A000872,1,Sequence 82 from patent US 7166769,alien,LAMP2.fasta,AAEDSQVGEGVVKIDCGGRCKGRCSKSSRPNLCLRACNSCCYRCNC...
LAMP2_L02A000612,L02A000612,LAMP2_L02A000612,1,Chrombacin,alien,LAMP2.fasta,AAEFPDFYDSEEQMGPHQEAEDEKDRADQRVLTEEEKKELENLAAM...


In [122]:
# removing synthetic
df = pd.read_csv('LAMP2_out.csv')

# entries with description patent are synthetic
fcontains = lambda x: 'patent' not in str(x)
dfnew = df[df.description.map(fcontains)]

dfnew.to_csv('LAMP2_out.csv')#! overwriting
# those with no descriptiuon cant be mapped
dfnew.head(2)


,id,name,AB,description,OX,dataset,seq
1,L02A000612,LAMP2_L02A000612,1,Chrombacin,alien,LAMP2.fasta,AAEFPDFYDSEEQMGPHQEAEDEKDRADQRVLTEEEKKELENLAAM...
2,L02A001914,LAMP2_L02A001914,1,"Ranacyclin-B-AL1 (frog, amphibians, animals)",alien,LAMP2.fasta,AAFRGCWTKNYSPKPCL


In [ ]:
#TODO http://yadamp.unisa.it
# cant access database

In [ ]:
#TODO http://phytamp.pfba-lab-tun.org/main.php
# blank page

In [34]:
#InverPep https://ciencias.medellin.unal.edu.co/gruposdeinvestigacion/prospeccionydisenobiomoleculas/InverPep/public/home_en
# no selection criteria on webpage

df = tocsv('InverPep.fasta', True, 'InverPep')
df.head(2)

input file InverPep.fasta
converting with fct:  <bound method converters.conv_InverPep of <__main__.converters object at 0x7fa7be0b9a50>>
['>AVP0722|-\n', 'NNLETTAFHRDDHET\n', '>BaAMPS_212|LF11-324\n', 'PFFWRIRIRR\n']
min, max seq:  6 300
saved df as csv to: InverPep_out.csv


,id,name,AB,description,OX,dataset,seq
-AVP0722,AVP0722,-AVP0722,1,None,alien,InverPep.fasta,NNLETTAFHRDDHET
LF11-324,BaAMPS_212,LF11-324,1,None,alien,InverPep.fasta,PFFWRIRIRR


In [3]:
# http://crdd.osdd.net/servers/hipdb/ no slection on webpage
#TODO
#df = tocsv('HIPdb_data.csv', True, 'HIPdb_data')
# file -I HIPdb_data.csv to check for inut type
#df = pd.read_csv('HIPdb_data.csv', encoding='utf-16le')

df = pd.read_excel('HIPdb_data.xls', sheet_name='Sheet1')
dfnew = df[df.SOURCE != 'synthetic']
dfnew = dfnew[['ID', 'PMID', 'INHIBITION/IC50', 'CELL LINE', 'SEQUENCE']]

dfnew.insert(2, 'AB', [1 for i in range(dfnew.shape[0]) ] )
dfnew.insert(5, 'dataset', [ 'HIPdb_data.xls' for i in range(dfnew.shape[0]) ] )

dfnew.columns = ['id', 'name', 'AB', 'description', 'OX', 'dataset', 'seq']
dfnew.head(2)


destin = 'HIPdb_data_out.csv'
dfnew.to_csv(destin, index = False )
print('saved df as csv to:', destin)

dfnew.head(2)

saved df as csv to: HIPdb_data_out.csv


,id,name,AB,description,OX,dataset,seq
0,HIP144,15113844,1,5,CEM/LAV-1,HIPdb_data.xls,AEAMSQVTN
1,HIP140,15113844,1,100,CEM/LAV-1,HIPdb_data.xls,AAAMSQVTN


In [ ]:
#Thiobase https://bioinfo-mml.sjtu.edu.cn/THIOBASE/index.php
#TODO not found

In [109]:
#EnzyBase http://biotechlab.fudan.edu.cn/database/EnzyBase/home.php
# no selection on webpage, file enzybase
df = tocsv('enzybase.fasta', True, 'enzybase')
df.head(2)

input file enzybase.fasta
converting with fct:  <bound method converters.conv_enzybase of <__main__.converters object at 0x7fd2d294cb20>>
['>EN47514126|AE|Q8HA43|phage B30 lysin,B30 lysin\n', 'MATYQEYKSRSNGNAYDIDGSFGAQCWDGYADYCKYLGLPYANCTNTGYARDIWEQRHENGILNYFDEVEVMQAGDVAIFMVVDGVTPYSHVAIFDSDAGGGYGWFLGQNQGGANGAYNLVKIPYSATYPTAFRPKSFKNAVTVTDNTGLNKGDYFIDVSAYQQADLTTTCQQAGTTKTIIKVSESIAWLSDRHQQQANTSDPIGYYHFGRFGGDSALAQREADLFLSNLPSKKVSYLVIDYEDSASADKQANTNAVIAFMDKIASAGYKPIYYSYKPFTLNNIDYQKIIAKYPNSIWIAGYPDYEVRTEPLWEFFPSMDGVRWWQFTSVGVAGGLDKNIVLLADDSSKMDIPKVDKPQELTFYQKLATNTKLDNSNVPYYEATLSTDYYVESKPNASSADKEFIKAGTRVRVYEKVNGWSRINHPESAQWVEDNYLVNATDM\n', '>EN61617816|D|O31982|N-acetylmuramoyl-L-alanine amidase BlyA,BlyA\n', 'MSVFTNSYIPVNKYTRPGLKLQGVKKCVLHYTANPGAGADNHRRYFSNAQVYASAHIFVDKAEAICIIPLNEVAYHANDIQQRDSAGNPYRGVAALKPNANFLSIGVEMCLEKDGSFHSDTVERTEDVFVELCNKFGLDPIDDIVRHYDITHKNCPAPWVSNSQKFVDFKNRVKAKMSGKSVSKASPTKPTTSSPSSSSAVSGSLKSKVDGLRFYSKPSWEDKDVVGTVNKGIGFPTVVEKVKVGSAYQYKVKNSKGTTYYITASDKYVDVTGSVKTSSSAPKTTST

,id,name,AB,description,OX,dataset,seq
EN47514126,Q8HA43,EN47514126,1,"type_AE_phage B30 lysin,B30 lysin",alien,enzybase.fasta,MATYQEYKSRSNGNAYDIDGSFGAQCWDGYADYCKYLGLPYANCTN...
EN61617816,O31982,EN61617816,1,type_D_N-acetylmuramoyl-L-alanine amidase BlyA...,alien,enzybase.fasta,MSVFTNSYIPVNKYTRPGLKLQGVKKCVLHYTANPGAGADNHRRYF...


In [ ]:
#Parapep http://crdd.osdd.net/raghava/parapep/
# no general download, if select for L chirality get df http://crdd.osdd.net/raghava/parapep/submitkey_target_search.php?ran=5411
# scrape: same format as .hr file

In [ ]:
# https://webs.iiitd.edu.in/raghava/antitbpdb/ select for peptide type = natural
# have to scrape https://webs.iiitd.edu.in/raghava/antitbpdb/browse_sub1.php?token=Natural&col=12

In [ ]:
# import from swissprot fasta, 
# anyone know list of metadata
# fingerprints
# unified dataset
# leraning algorithms

# ? non secretary proteins, how to identify?

# NEGATIVE SAMPLES